This notebook generates the data for the campaign - unfortunately the original dataset is a little disjointed. But we are going to use batch inference to fix that and make a very strong dataset for analysis.

This notebook works with Serverless

In [0]:
%pip install dbdemos

In [0]:
import dbdemos
dbdemos.install(
  'aibi-marketing-campaign', 
  catalog='shm', 
  schema='marketing'
  )

We are going to move over the Genie space from the aibi-marketing-campaign folder into the new workspace and delete the dbdemo folder. We are also going to grab 1000 rows from the Bright Data Best Buy product dataset and use it to generate better reviews using AI_QUERY.

TODO: I did this manually buy clean it up

We can use AI Query to make a MUCH better campaign table using this product table. We are going to use structured outputs to generate a useful json for a random sample of the products. This query is a bit of a beast but randomly joins 100 products to our campaign table and uses Llama 70B to generate solid campaign slogans for real products

In [0]:
%sql
CREATE OR REPLACE TABLE shm.marketing.campaigns AS
-- Get a nice joined table
WITH RandomProduct AS (
  SELECT
      ROW_NUMBER() OVER (ORDER BY RAND()) + 100 AS row,
      *
  FROM shm.marketing.products
  LIMIT 100
)
SELECT 
  C.campaign_id, C.template, C.cost, 
  C.start_date, C.end_date, C.mailing_list,
  P.title, P.product_id, 
  json_tuple(
    AI_QUERY(
      'databricks-meta-llama-3-3-70b-instruct',
      CONCAT('Using the following product title, description, and feature summary, make a witty campaign name, campaign description, and email marketing subject line. Return them in json format. \n', title, product_description, features_summary),
      responseFormat => '{
        "type": "json_schema",
        "json_schema": {
          "name": "generated_campaign",
          "schema": {
            "type": "object",
            "properties": {
              "campaign_name": {"type": "string"},
              "campaign_description": {"type": "string"},
              "subject_line": {"type": "string"}
            }
          }
        }
      }'
    ), 
    'campaign_name', 'campaign_description', 'subject_line') 
    AS (campaign_name, campaign_description, subject_line)
FROM RandomProduct P
JOIN shm.marketing.campaigns C
ON P.row = C.campaign_id

In [0]:
%sql
SELECT * FROM shm.marketing.campaigns
LIMIT 1